# Homework 10: `SQL`

## Due Date:  Thursday, November 16th at 11:59 PM

You will create a database of the NASA polynomial coefficients for each specie.

**Please turn in your database with your `Jupyter` notebook!**

# Question 1: Convert XML to a SQL database

Create two tables named `LOW` and `HIGH`, each corresponding to data given for the low and high temperature range.
Each should have the following column names:

- `SPECIES_NAME`
- `TLOW`
- `THIGH`
- `COEFF_1`
- `COEFF_2`
- `COEFF_3`
- `COEFF_4`
- `COEFF_5`
- `COEFF_6`
- `COEFF_7`

Populate the tables using the XML file you created in last assignment. If you did not complete the last assignment, you may also use the `example_thermo.xml` file.

`TLOW` should refer to the temperature at the low range and `THIGH` should refer to the temperature at the high range.  For example, in the `LOW` table, $H$ would have `TLOW` at $200$ and `THIGH` at $1000$ and in the `HIGH` table, $H$ would have `TLOW` at $1000$ and `THIGH` at $3500$.

For both tables, `COEFF_1` through `COEFF_7` should be populated with the corresponding coefficients for the low temperature data and high temperature data.

In [45]:
#Import SQLlite
import sqlite3
import numpy as np
import pandas as pd

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('HW10_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS LOW")
cursor.execute("DROP TABLE IF EXISTS HIGH")
cursor.execute("PRAGMA foreign_keys=1")

#Create High and Low tables
cursor.execute('''CREATE TABLE LOW ( 
               SPECIES_NAME TEXT NOT NULL, 
               TLOW REAL NOT NULL, 
               THIGH REAL NOT NULL, 
               COEFF_1 TEXT NOT NULL,
               COEFF_2 TEXT NOT NULL,
               COEFF_3 TEXT NOT NULL,
               COEFF_4 TEXT NOT NULL,
               COEFF_5 TEXT NOT NULL,
               COEFF_6 TEXT NOT NULL,
               COEFF_7 TEXT NOT NULL)''')

# Commit changes to the database
db.commit()
cursor.execute('''CREATE TABLE HIGH (
               SPECIES_NAME TEXT NOT NULL, 
               TLOW REAL NOT NULL, 
               THIGH REAL NOT NULL, 
               COEFF_1 TEXT NOT NULL,
               COEFF_2 TEXT NOT NULL,
               COEFF_3 TEXT NOT NULL,
               COEFF_4 TEXT NOT NULL,
               COEFF_5 TEXT NOT NULL,
               COEFF_6 TEXT NOT NULL,
               COEFF_7 TEXT NOT NULL)''')
db.commit()


In [46]:
#Parse XML to get info for each species
import xml.etree.ElementTree as ET

#Get the xml
tree = ET.parse('thermo.xml')
root = tree.getroot()

#get species
species = root.find('speciesData').findall('species')

for specie in species:
    name = specie.get('name')
    
    #get low temp high/low and coeffs for each specie
    NASA = specie.find('thermo').findall('NASA')
    
    #get low info
    low_tmax = NASA[0].get('Tmax')
    low_tmin = NASA[0].get('Tmin')
    Low_C_1,Low_C_2,Low_C_3,Low_C_4,Low_C_5,Low_C_6,Low_C_7 = NASA[0].find('floatArray').text.split()
    lows_to_insert = (name,float(low_tmin),float(low_tmax),Low_C_1,Low_C_2,Low_C_3,Low_C_4,Low_C_5,Low_C_6,Low_C_7)
    
    
    #get low info
    high_tmax = NASA[1].get('Tmax')
    high_tmin = NASA[1].get('Tmin')
    High_C_1,High_C_2,High_C_3,High_C_4,High_C_5,High_C_6,High_C_7 = NASA[1].find('floatArray').text.split()
    high_to_insert = name,float(high_tmin),float(high_tmax),High_C_1,High_C_2,High_C_3,High_C_4,High_C_5,High_C_6,High_C_7
    
    #Insert the values for each species into table
    cursor.execute('''INSERT INTO LOW 
                  (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', lows_to_insert)
    cursor.execute('''INSERT INTO HIGH 
                  (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', high_to_insert)

In [47]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

In [48]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(LOW)")]
query = '''SELECT * FROM LOW'''
viz_tables(candidate_cols, query)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,200.0,1000.0,"3.16826710E+00,","-3.27931884E-03,","6.64306396E-06,","-6.12806624E-09,","2.11265971E-12,","2.91222592E+04,","2.05193346E+00,"
1,O2,200.0,1000.0,"3.78245636E+00,","-2.99673416E-03,","9.84730201E-06,","-9.68129509E-09,","3.24372837E-12,","-1.06394356E+03,","3.65767573E+00,"
2,H,200.0,1000.0,"2.50000000E+00,","7.05332819E-13,","-1.99591964E-15,","2.30081632E-18,","-9.27732332E-22,","2.54736599E+04,","-4.46682853E-01,"
3,H2,200.0,1000.0,"2.34433112E+00,","7.98052075E-03,","-1.94781510E-05,","2.01572094E-08,","-7.37611761E-12,","-9.17935173E+02,","6.83010238E-01,"
4,OH,200.0,1000.0,"3.99201543E+00,","-2.40131752E-03,","4.61793841E-06,","-3.88113333E-09,","1.36411470E-12,","3.61508056E+03,","-1.03925458E-01,"
5,H2O,200.0,1000.0,"4.19864056E+00,","-2.03643410E-03,","6.52040211E-06,","-5.48797062E-09,","1.77197817E-12,","-3.02937267E+04,","-8.49032208E-01,"
6,HO2,200.0,1000.0,"4.30179801E+00,","-4.74912051E-03,","2.11582891E-05,","-2.42763894E-08,","9.29225124E-12,","2.94808040E+02,","3.71666245E+00,"
7,H2O2,200.0,1000.0,"4.27611269E+00,","-5.42822417E-04,","1.67335701E-05,","-2.15770813E-08,","8.62454363E-12,","-1.77025821E+04,","3.43505074E+00,"


In [49]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
query = '''SELECT * FROM HIGH'''
viz_tables(candidate_cols, query)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,1000.0,3500.0,"2.56942078E+00,","-8.59741137E-05,","4.19484589E-08,","-1.00177799E-11,","1.22833691E-15,","2.92175791E+04,","4.78433864E+00,"
1,O2,1000.0,3500.0,"3.28253784E+00,","1.48308754E-03,","-7.57966669E-07,","2.09470555E-10,","-2.16717794E-14,","-1.08845772E+03,","5.45323129E+00,"
2,H,1000.0,3500.0,"2.50000001E+00,","-2.30842973E-11,","1.61561948E-14,","-4.73515235E-18,","4.98197357E-22,","2.54736599E+04,","-4.46682914E-01,"
3,H2,1000.0,3500.0,"3.33727920E+00,","-4.94024731E-05,","4.99456778E-07,","-1.79566394E-10,","2.00255376E-14,","-9.50158922E+02,","-3.20502331E+00,"
4,OH,1000.0,3500.0,"3.09288767E+00,","5.48429716E-04,","1.26505228E-07,","-8.79461556E-11,","1.17412376E-14,","3.85865700E+03,","4.47669610E+00,"
5,H2O,1000.0,3500.0,"3.03399249E+00,","2.17691804E-03,","-1.64072518E-07,","-9.70419870E-11,","1.68200992E-14,","-3.00042971E+04,","4.96677010E+00,"
6,HO2,1000.0,3500.0,"4.01721090E+00,","2.23982013E-03,","-6.33658150E-07,","1.14246370E-10,","-1.07908535E-14,","1.11856713E+02,","3.78510215E+00,"
7,H2O2,1000.0,3500.0,"4.16500285E+00,","4.90831694E-03,","-1.90139225E-06,","3.71185986E-10,","-2.87908305E-14,","-1.78617877E+04,","2.91615662E+00,"


# Question 2: `WHERE` Statements

1. Write a `Python` function `get_coeffs` that returns an array of 7 coefficients.  
   
   The function should take in two parameters: 1.) `species_name` and 2.) `temp_range`, an indicator variable ('low' or 'high') to indicate whether the coefficients should come from the low or high temperature range.  
   The function should use `SQL` commands and `WHERE` statements on the table you just created in Question 1 (rather than taking data from the XML directly).
```python
def get_coeffs(species_name, temp_range):
    ''' Fill in here'''
    return coeffs
```

2. Write a python function `get_species` that returns all species that have a temperature range above or below a given value. The function should take in two parameters: 1.) `temp` and 2.) `temp_range`, an indicator variable ('low' or 'high').

  When temp_range is 'low', we are looking for species with a temperature range lower than the given temperature, and for a 'high' temp_range, we want species with a temperature range higher than the given temperature.

  This exercise may be useful if different species have different `LOW` and `HIGH` ranges.

  And as before, you should accomplish this through `SQL` queries and where statements.

```python
def get_species(temp, temp_range):
    ''' Fill in here'''
    return coeffs
```

In [104]:
#Question 2 Part 1
def get_coeffs(species_name, temp_range):
    #get list of species
    query = '''SELECT SPECIES_NAME FROM LOW'''
    species_list = cursor.execute(query).fetchall()
    species_list = [s[0].strip(',') for s in species_list]
    #check that specie_name inputed it correct type of species
    if species_name not in species_list:
        raise ValueError("Input must be 'O','O2','H','H2','OH','H2O','HO2','H2O2' for species_name")
    #get the coeffs based on temp range
    if temp_range == 'low':
        query = '''SELECT COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7 FROM LOW WHERE SPECIES_NAME = ? '''
        coeffs = cursor.execute(query,species_name).fetchall()
    elif temp_range == 'high': 
        query = '''SELECT COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7 FROM HIGH WHERE SPECIES_NAME = ? '''
        coeffs = cursor.execute(query,species_name).fetchall()
    else: #make sure temp range is correctly inputed
        raise ValueError("Must input 'low' or 'high' for temp_range")
    return coeffs

In [115]:
# get_coeffs function in action
print("return type: ",type(get_coeffs('O','high')))
print(get_coeffs('O','high'))

return type:  <class 'list'>
[('2.56942078E+00,', '-8.59741137E-05,', '4.19484589E-08,', '-1.00177799E-11,', '1.22833691E-15,', '2.92175791E+04,', '4.78433864E+00,')]


In [116]:
def get_species(temp, temp_range):
    if type(temp) != int and type(temp) != float:
        raise ValueError("Must input a number' for temp")
    #get the coeffs based on temp range
    if temp_range == 'low':
        query = '''SELECT SPECIES_NAME FROM LOW WHERE TLOW < ?'''
        q = cursor.execute(query,(temp,)).fetchall()
    elif temp_range == 'high': 
        query = '''SELECT SPECIES_NAME FROM HIGH WHERE THIGH > ?'''
        q = cursor.execute(query,(temp,)).fetchall()
    else: #make sure temp range is correctly inputed
        raise ValueError("Must input 'low' or 'high' for temp_range")
    return q

In [117]:
# get_species function in action
print(get_species(300,'low'))
print(get_species(100,'low'))
print(get_species(3000,'high'))

[('O',), ('O2',), ('H',), ('H2',), ('OH',), ('H2O',), ('HO2',), ('H2O2',)]
[]
[('O',), ('O2',), ('H',), ('H2',), ('OH',), ('H2O',), ('HO2',), ('H2O2',)]


# Question 3: `JOIN` STATEMENTS

Create a table named `ALL_TEMPS` that has the following columns:

- `SPECIES_NAME`
- `TEMP_LOW`
- `TEMP_HIGH`

This table should be created by joining the tables `LOW` and `HIGH` on the value `SPECIES_NAME`.

1. Write a `Python` function `get_range` that returns the range of temperatures for a given species_name.

The range should be computed within the `SQL` query (i.e. you should subtract within the `SELECT` statement in the `SQL` query).
```python
def get_range(species_name):
    '''Fill in here'''
    return range
```

Note that `TEMP_LOW` is the lowest temperature in the `LOW` range and `TEMP_HIGH` is the highest temperature in the `HIGH` range.

In [ ]:
db.close()